##  Dataset : California Housing Prices

https://www.kaggle.com/datasets/camnugent/california-housing-prices

### Variables disponibles :
- `longitude`, `latitude` : Coordonnées géographiques
- `housing_median_age` : Âge médian des maisons (années)
- `total_rooms` : Nombre total de pièces dans le district
- `total_bedrooms` : Nombre total de chambres dans le district
- `population` : Population du district
- `households` : Nombre de ménages dans le district
- `median_income` : Revenu médian des ménages (en 10k$)
- `median_house_value` : Valeur médiane des maisons (en $)
- `ocean_proximity` : Proximité de l'océan (NEAR BAY, <1H OCEAN, INLAND, NEAR OCEAN, ISLAND)

In [4]:
# Créer la session spark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.master("local").getOrCreate()
sc = spark.sparkContext

### Exercice 1.1 : Statistiques globales

In [7]:
# Charger le dataset
df = spark.read.options(header=True, inferSchema=True, sep=",", encoding='UTF-8').csv("../Data/housing.csv")
# À FAIRE :
# 1. Afficher le nombre total de lignes
df.count()
# 2. Calculer la valeur moyenne des maisons
df.select(avg('median_house_value')).show()
# 3. Trouver le revenu médian le plus élevé
df.select(max('median_income')).show()
# 4. Identifier l'âge moyen des maisons
df.select(avg('housing_median_age')).show()

+-----------------------+
|avg(median_house_value)|
+-----------------------+
|     206855.81690891474|
+-----------------------+

+------------------+
|max(median_income)|
+------------------+
|           15.0001|
+------------------+

+-----------------------+
|avg(housing_median_age)|
+-----------------------+
|     28.639486434108527|
+-----------------------+



### Exercice 1.2 : Statistiques par variable

In [11]:
# À FAIRE :
# 1. Calculer min, max, moyenne pour median_house_value
df.select(
    min("median_house_value").alias("min_value"),
    max("median_house_value").alias("max_value"),
    avg("median_house_value").alias("avg_value")
).show()
# 2. Calculer min, max, moyenne pour median_income  
df.select(
    min("median_income").alias("min_value"),
    max("median_income").alias("max_value"),
    avg("median_income").alias("avg_value")
).show()
# 3. Calculer min, max, moyenne pour housing_median_age
df.select(
    min("housing_median_age").alias("min_value"),
    max("housing_median_age").alias("max_value"),
    avg("housing_median_age").alias("avg_value")
).show()
# 4. Compter combien de districts ont une population > 5000
df.filter(col("population") > 5000).count()

+---------+---------+------------------+
|min_value|max_value|         avg_value|
+---------+---------+------------------+
|  14999.0| 500001.0|206855.81690891474|
+---------+---------+------------------+

+---------+---------+------------------+
|min_value|max_value|         avg_value|
+---------+---------+------------------+
|   0.4999|  15.0001|3.8706710029070246|
+---------+---------+------------------+

+---------+---------+------------------+
|min_value|max_value|         avg_value|
+---------+---------+------------------+
|      1.0|     52.0|28.639486434108527|
+---------+---------+------------------+



305

### Exercice 1.3 : Distribution des catégories

In [29]:
# À FAIRE :
# 1. Compter combien de districts par type de proximité océan
ocean_count = df.groupBy("ocean_proximity").count()
# 2. Calculer le pourcentage de chaque catégorie
total_district = df.count()
ocean_count.withColumn("percentage", round((col("count") / total_district) *100, 2)).sort(desc("percentage")).show()
most_frequent = ocean_count.withColumn("percentage", round((col("count") / total_district) *100, 2)).sort(desc("percentage")).first()

# 3. Identifier la catégorie la plus fréquente
print(f"Catégorie la plus fréquente : {most_frequent[0]}")

+---------------+-----+----------+
|ocean_proximity|count|percentage|
+---------------+-----+----------+
|      <1H OCEAN| 9136|     44.26|
|         INLAND| 6551|     31.74|
|     NEAR OCEAN| 2658|     12.88|
|       NEAR BAY| 2290|     11.09|
|         ISLAND|    5|      0.02|
+---------------+-----+----------+

Catégorie la plus fréquente : <1H OCEAN


**Questions :**
- Quel est le prix moyen d'une maison en Californie ?
- Quelle est la répartition géographique par proximité océan ?

### Exercice 2.1 : Comparaisons par proximité océan

In [30]:
# À FAIRE :
# 1. Prix moyen des maisons par type de proximité océan
# 2. Revenu moyen par type de proximité océan
# 3. Âge moyen des maisons par type de proximité océan
# 4. Population moyenne par type de proximité océan
df.groupBy("ocean_proximity").agg(
    avg("median_house_value").alias("prix_moyen"),
    avg("median_income").alias("revenu_moyen"),
    avg("housing_median_age").alias("age_moyen"),
    avg("population").alias("population_moyenne")
).orderBy(col("prix_moyen").desc()).show()

+---------------+------------------+------------------+------------------+------------------+
|ocean_proximity|        prix_moyen|      revenu_moyen|         age_moyen|population_moyenne|
+---------------+------------------+------------------+------------------+------------------+
|         ISLAND|          380440.0|2.7444200000000003|              42.4|             668.0|
|       NEAR BAY|259212.31179039303| 4.172884759825336| 37.73013100436681|1230.3174672489083|
|     NEAR OCEAN|249433.97742663656| 4.005784800601957|29.347253574115875|1354.0086531226486|
|      <1H OCEAN|240084.28546409807|4.2306819176882655|29.279225043782837|1520.2904991243433|
|         INLAND|124805.39200122119| 3.208996382231716| 24.27186689055106|1391.0462524805373|
+---------------+------------------+------------------+------------------+------------------+



### Exercice 2.2 : Segmentation par revenus

In [32]:
# À FAIRE :
# 1. Créer 3 segments de revenus :
#    - Revenus faibles : median_income < 3.0
#    - Revenus moyens : 3.0 <= median_income < 6.0  
#    - Revenus élevés : median_income >= 6.0
# 2. Calculer le prix moyen des maisons par segment
# 3. Compter le nombre de districts par segment

df_segment = df.withColumn("segment_revenu", when(col("median_income") < 3.0, "revenus faibles")
              .when((col("median_income") >= 3.0) & (col("median_income") < 6.0), "revenus moyens")
              .otherwise("revenus élevés"))

df_segment.groupBy("segment_revenu").agg(
    avg("median_house_value").alias("prix_moyen"),
    count("*").alias("nb_districts")
).show()

+---------------+------------------+------------+
| segment_revenu|        prix_moyen|nb_districts|
+---------------+------------------+------------+
| revenus élevés| 378206.7669616519|        2373|
| revenus moyens|218965.56310768667|       10902|
|revenus faibles|133721.22661235573|        7365|
+---------------+------------------+------------+



### Exercice 2.3 : Segmentation par âge des maisons

In [33]:
# À FAIRE :
# 1. Créer 3 catégories d'âge :
#    - Récentes : housing_median_age < 15
#    - Moyennes : 15 <= housing_median_age < 35
#    - Anciennes : housing_median_age >= 35
# 2. Analyser le prix moyen par catégorie d'âge
# 3. Analyser la population moyenne par catégorie d'âge
df_age = df.withColumn("categorie_age", 
                when(col("housing_median_age") < 15, "Récentes")
                .when((col("housing_median_age") >= 15) & (col("housing_median_age") < 35), "Moyennes")
                .otherwise("revenus Anciennes"))

df_age.groupBy("categorie_age").agg(
    avg("median_house_value").alias("prix_moyen"),
    avg("population").alias("population_moyenne")
).show()

+-----------------+------------------+------------------+
|    categorie_age|        prix_moyen|population_moyenne|
+-----------------+------------------+------------------+
|         Moyennes|202377.17341310647| 1493.496120407591|
|revenus Anciennes|218455.61746651787| 1063.733677455357|
|         Récentes|194156.96252252252|2097.6825225225225|
+-----------------+------------------+------------------+



**Questions :**
- Les maisons près de l'océan sont-elles vraiment plus chères ?
- Y a-t-il une corrélation entre revenus élevés et prix des maisons ?

### Exercice 3.1 : Calcul de ratios

In [36]:
# À FAIRE :
# 1. Calculer rooms_per_household = total_rooms / households
# 2. Calculer bedrooms_per_room = total_bedrooms / total_rooms  
# 3. Calculer population_per_household = population / households
# 4. Afficher les statistiques (min, max, moyenne) de ces nouveaux ratios
df_ratios = df.withColumn(
    "rooms_per_household", col("total_rooms") / col("households")
).withColumn(
    "bedrooms_per_room", col("total_bedrooms") / col("total_rooms")
).withColumn(
    "population_per_household", col("population") / col("households")
)

df_ratios.select(
    # rooms_per_household
    min("rooms_per_household").alias("min_rooms_per_household"),
    max("rooms_per_household").alias("max_rooms_per_household"),
    avg("rooms_per_household").alias("avg_rooms_per_household"),

    # bedrooms_per_room
    min("bedrooms_per_room").alias("min_bedrooms_per_room"),
    max("bedrooms_per_room").alias("max_bedrooms_per_room"),
    avg("bedrooms_per_room").alias("avg_bedrooms_per_room"),
    # population_per_household
    
    min("population_per_household").alias("min_population_per_household"),
    max("population_per_household").alias("max_population_per_household"),
    avg("population_per_household").alias("avg_population_per_household"),
).show()

+-----------------------+-----------------------+-----------------------+---------------------+---------------------+---------------------+----------------------------+----------------------------+----------------------------+
|min_rooms_per_household|max_rooms_per_household|avg_rooms_per_household|min_bedrooms_per_room|max_bedrooms_per_room|avg_bedrooms_per_room|min_population_per_household|max_population_per_household|avg_population_per_household|
+-----------------------+-----------------------+-----------------------+---------------------+---------------------+---------------------+----------------------------+----------------------------+----------------------------+
|     0.8461538461538461|      141.9090909090909|      5.428999742190365|                  0.1|                  1.0|  0.21303883048085015|          0.6923076923076923|          1243.3333333333333|           3.070655159436382|
+-----------------------+-----------------------+-----------------------+-------------------

### Exercice 3.2 : Analyse des ratios par zone

In [42]:
# À FAIRE :
# 1. Calculer la moyenne de rooms_per_household par proximité océan
# 2. Calculer la moyenne de bedrooms_per_room par proximité océan
# 3. Identifier quelle zone a les maisons les plus spacieuses
ratio_ocean = df_ratios.groupBy("ocean_proximity").agg(
    # 1.
    avg("rooms_per_household").alias("avg_rooms_per_household"),
    # 2.
    avg("bedrooms_per_room").alias("avg_bedrooms_per_room")
).orderBy(col("avg_rooms_per_household").desc())

ratio_ocean.show()

most_spacious = ratio_ocean.collect()[0]
print(f"Zone la plus spacieuse : {most_spacious['ocean_proximity']}")

+---------------+-----------------------+---------------------+
|ocean_proximity|avg_rooms_per_household|avg_bedrooms_per_room|
+---------------+-----------------------+---------------------+
|         INLAND|      5.977265079384676|    0.203337883647453|
|         ISLAND|       5.65657716330408|   0.2732464411449803|
|       NEAR BAY|       5.22170524136999|   0.2133680317685574|
|     NEAR OCEAN|      5.206007549366366|  0.21910028911609664|
|      <1H OCEAN|     5.1525760933659015|  0.21813507935841644|
+---------------+-----------------------+---------------------+

Zone la plus spacieuse : INLAND


### Exercice 3.3 : Indicateurs de densité

In [46]:
# À FAIRE :
# 1. Calculer la densité = population / households
# 2. Identifier les districts les plus denses (top 10)
# 3. Analyser le prix moyen des districts très denses vs peu denses
df_density = df.withColumn("densite", col("population") / col("households"))

df_density.orderBy(col("densite").desc()).limit(10).show()

df_density_analyse = df_density. withColumn(
    "categorie_densite",
    when(col("densite") >= 4.0, "Très dense")
    .when(col("densite") >= 3.0, "Dense")
    .when(col("densite") >= 2.0, "Moyen")
    .otherwise("Peu dense")
)

df_density_analyse.groupBy("categorie_densite").agg(
    avg("median_house_value").alias("prix_moyen")
).orderBy(col("prix_moyen").desc()).show()



+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|           densite|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+------------------+
|  -121.98|   38.32|              45.0|       19.0|           5.0|    7460.0|       6.0|      10.2264|          137500.0|         INLAND|1243.3333333333333|
|  -120.51|   40.41|              36.0|       36.0|           8.0|    4198.0|       7.0|       5.5179|           67500.0|         INLAND| 599.7142857142857|
|   -120.7|   35.32|              46.0|      118.0|          17.0|    6532.0|      13.0|       4.2639|          350000.0|     NEAR OCEAN|502.46153846153845|
|  -121.15|   38.69|              52.0|      240.0|       

**Questions :**
- Quelles zones offrent le plus d'espace par ménage ?
- La densité de population impacte-t-elle les prix ?